## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 아래 4가지의 도구를 사용하고 있습니다.
- 필터 query로 데이터베이스 정보 추출(추천을 위함): chroma를 활용해 수집된 wine database에 필터를 적용하여 특정 와인만 뽑아올 수 있습니다.    
    - input: {body:3, price:100000~120000}
    - output: [Document(page_content='포지오 데이 젤시', metadata={'price': '30000', ...}),]
- 자연어 query로 데이터베이스 정보 추출(추천을 위함): DocArrayInMemorySearch을 활용해 wine database에 자연어로 특정 데이터를 뽑아 올 수 있습니다.
    - input: "프랑스에서 만들어진 와인은?"
- 특정 URL query로 웹 페이지에서 정보 추출(와인 구매링크에서 정보를 가져올 때 사용): SeleniumURLLoader를 이용해 특정 url로 부터 웹페이지 정보를 얻을 수 있습니다.
- 자연어 구글 검색으로 웹 페이지에서 정보 추출(Q&A처럼 그 외 검색이 필요한 경우 사용): SerpAPI를 이용해 구글에서 검색 결과를 가져올 수 있습니다.

#### 당신의 목표는 아래와 같습니다.
1. 다양한 도구를 사용하거나 알고리즘을 탐구하여 데이터베이스를 연결할 수 있는 좋은 방식을 찾습니다. (예를 들면 와인바의 상세정보를 검색하기 위한 카카오 위치 정보 API등을 추가할 수 있을 듯 합니다.)
2. 도구를 사용할 때는 적절한 format을 사용하는 것이 성능에 영향을 줍니다. 각 도구들에 대한 적절한 input format을 찾습니다.


#### 참고사항
현재 말이 너무 모호하게 써있고, 도구의 사용처가 확실하지 않습니다. 이를 구체화하는 과정이 필요하기도 합니다.

-------------------------------------------------------------------------------

### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [5]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
kakao_api_key = config['KAKAO_MAP']['KAKAO_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

In [6]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import Chroma

### Load wine database json

In [7]:
df = pd.read_json('./data/unified_wine_data.json', encoding='utf-8', lines=True)

In [8]:
df.head()

,url,site_name,price,name,en_name,img_url,body,acidity,tannin,sweetness,alcohol,wine_type,country,grape,rating,pickup_location,vivino_link,flavor_description,pairing
0,https://www.winenara.com/shop/product/product_...,winenara,38800,샹동 로제 브룻,Chandon Rose brut,https://www.winenara.com/uploads/product/550/8...,-1,,,-1,-1,로제,호주,,3.7,,https://www.vivino.com/US-CA/en/chandon-califo...,로제 와인의 생동감을 더한 우아한 스파클링 와인으로 입 안을 가득 채우는 체리파이 ...,
1,https://www.winenara.com/shop/product/product_...,winenara,200000,티터 토터 카베르네소비뇽,TEETER-TOTTER CABERNET SAUVIGNON,https://www.winenara.com/uploads/product/550/4...,4,,,-1,-1,레드,미국,,4.3,,https://www.vivino.com/US-CA/en/teeter-totter-...,"첫 향으로 블랙베리, 라즈베리, 바닐라 향이 튀어 오르며 뒤이어 정향, 제비꽃, 라...","피자, 스테이크"
2,https://www.winenara.com/shop/product/product_...,winenara,1100000,아르망 드 브리냑 브룻 골드,Armand De Brignac Brut Gold,https://www.winenara.com/uploads/product/550/5...,-1,,,2,-1,스파클링,프랑스,,4.4,,https://www.vivino.com/US-CA/en/armand-de-brig...,"전통 샴페인의 진수를 보여주는 와인으로, 복합적이며 풀바디한 느낌이 압도적입니다. ...","기름진 생선 요리, 돼지고기 요리"
3,https://www.winenara.com/shop/product/product_...,winenara,595000,크룩 로제,Krug brut Rose,https://www.winenara.com/uploads/product/550/0...,-1,,,-1,-1,로제,프랑스,,4.6,,https://www.vivino.com/US-CA/en/krug-brut-rose...,,
4,https://www.winenara.com/shop/product/product_...,winenara,100000,도멘 퐁사르 슈발리에 마랑지 1er 라 푸시에,DOMAINE PONSARD CHEVALIER MARANGES 1ER LA FUSS...,https://www.winenara.com/uploads/product/550/1...,3,,,-1,-1,레드,프랑스,,4.2,,https://www.vivino.com/US-CA/en/ponsard-cheval...,밝은 가넷 컬러가 아름다우며 잘 익은 체리와 자두 등 핵과 열매의 아로마와 함께 잘...,


### Prepare Langchain Tool

#### Tool1: Wine database 1

In [9]:
df['page_content'] = ''
columns = ['name', 'pairing']
for column in columns:
    if column != 'page_content':
        df['page_content'] += column + ':' + df[column].astype(str) + ','

In [10]:
columns = ['rating', 'price', 'body', 'sweetness', 'alcohol', 'acidity', 'tannin']
for idx in df.index:
    for column in columns:
        if type(df[column][idx]) == str:
            df[column][idx] = df[column][idx].replace(',', '')
        df[column][idx] = float(df[column][idx]) if df[column][idx] != '' else -1

In [11]:
df = df.drop(columns=['name', 'pairing'])
df = df.sort_values('rating', ascending=False) # rating이 높은 순서대로 정렬

In [21]:
loader =DataFrameLoader(data_frame=df, page_content_column='page_content')
docs = loader.load()
embeddings = OpenAIEmbeddings()

아래는 wine database1에 metadata_field Attribute이다. 아래를 기준으로 서치를 진행하게 된다.

In [14]:
metadata_field_info = [
    AttributeInfo(
        name="body",
        description="1-5 rating for the body of wine",
        type="int",
    ),
    AttributeInfo(
        name="tannin",
        description="1-5 rating for the tannin of wine",
        type="int",
    ),
    AttributeInfo(
        name="sweetness",
        description="1-5 rating for the sweetness of wine",
        type="int",
    ),
    AttributeInfo(
        name="alcohol",
        description="1-5 rating for the alcohol of wine",
        type="int",
    ),
    AttributeInfo(
        name="price",
        description="The price of the wine",
        type="int",
    ),
    AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine", 
        type="float"
    ),
    AttributeInfo(
        name="wine_type", 
        description="The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'", 
        type="string"
    ),
    AttributeInfo(
        name="country", 
        description="The country of wine. It can be '기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주'", 
        type="float"
    ),
]

In [ ]:
vectorstore = Chroma.from_documents(docs, embeddings)

In [43]:
document_content_description = "A database of wines. 'name' and 'pairing' must be included in the query, and 'Body', 'Tannin', 'Sweetness', 'Alcohol', 'Price', 'Rating', 'Wine_Type', and 'Country' can be included in the filter. query and filter must be form of 'key: value'. For example, query: 'name: 돔페리뇽, pairing:육류'."
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)  # Added missing closing parenthesis


In [46]:
retriever.get_relevant_documents('pairing:육류') # gt means greater than, lt means less than, eq means equal to

query='pairing:육류' filter=None limit=None


[Document(page_content='name:멈 그랑 꼬르똥,pairing:육류 신선한 해물요리,', metadata={'url': 'https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000001577', 'site_name': 'wine_and_more', 'price': 89000.0, 'en_name': 'MUMM Mumm Grand Cordon G', 'img_url': 'https://github.com/audreyaiai/chatwine-dev/blob/main/data/wine_and_more/img/멈_그랑_꼬르똥.png?raw=True', 'body': -1, 'acidity': -1, 'tannin': -1, 'sweetness': -1, 'alcohol': -1, 'wine_type': '', 'country': '', 'grape': '', 'rating': -1, 'pickup_location': '와인앤모어 광화문점,와인앤모어 삼성1호점,와인앤모어 서래점,와인앤모어 서울대입구역점,와인앤모어 스타필드시티 명지점,와인앤모어 역삼센터필드점,와인앤모어 학동역점', 'vivino_link': '', 'flavor_description': ".H.MUMM CHAMPAGNE 축하의 순간을 함께하는 샴페인 샴페인 하우스 멈은 축하의 아이콘이자 가장 아방가르드한 샴페인으로 '오직 최고만을 추구한다'는 멈의 . 모토를 현대적으로 재해석하며 완벽함을 추구한다. 특히, 멈 그랑 꼬르동 제품은 Red Sash가 병에 입체적으로 새겨져있는 유니크한 바틀 디자인으로 혁신적이고 대담함을 드러낸다. 한 모금을 머금는 순간, 입 안 가득 풍부한 버블, 신선한 복숭아, 살구 등의 과일과 바닐라, 견과류의 느낌이 결합되어 끝 맛의 응축도를 더해준다."}),
 Document(page_content='name:페트라 징가리,pairing:육류 스테이크 살라미 경성치즈류,', metadata={'url': 'h

In [19]:
df_new.tail()

,url,site_name,price,en_name,img_url,body,acidity,tannin,sweetness,alcohol,wine_type,country,grape,rating,pickup_location,vivino_link,flavor_description,page_content
1192,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,20000.0,Wabi Sabi Pet Nat,https://github.com/audreyaiai/chatwine-dev/blo...,-1,-1,-1,-1,-1,,,,-1,"SSG푸드마켓 도곡점,와인앤모어 AK광명점,와인앤모어 교대역점,와인앤모어 논현점,와...",,아로마틱한 과실의 촘촘한 구조감의 내추럴 스파클링 와인 Kremstal 지역의 역암...,"name:[클리어런스] 와비 사비 펫 낫,pairing:리조또 흰 육류,"
1191,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,29900.0,Chateau de Laborde CHATEAU DE LABORDE Annonali...,https://github.com/audreyaiai/chatwine-dev/blo...,-1,-1,-1,-1,-1,,,,-1,"와인앤모어 청담점,와인앤모어 합정역점",,"45년 올드바인 포도로 만든 와인 체리 색을 띠고 있고 블랙 체리, 페퍼 등 동양적...","name:샤또 드 라보르드 부르고뉴 피노누아 뀌베 에이치,pairing:안심, 등심..."
1190,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,105000.0,Chateau Lynch Moussas LYNCH,https://github.com/audreyaiai/chatwine-dev/blo...,-1,-1,-1,-1,-1,,,,-1,"와인앤모어 AK광명점,와인앤모어 광교점,와인앤모어 교대역점,와인앤모어 동판교점,와인...",,-MOUSSES PAUILLAC 부드럽고 차분한 개성있는 와인 짙은 루비색을 띤 개...,"name:샤또 랭쉬 무싸,pairing:양고기 치즈 그릴에 구운 육류요리,"
1189,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,78000.0,Wolf Blass Gold Label WOLF BLASS Gold Label Re...,https://github.com/audreyaiai/chatwine-dev/blo...,-1,-1,-1,-1,-1,,,,-1,"와인앤모어 광교점,와인앤모어 군자역점,와인앤모어 다산점,와인앤모어 서래점,와인앤모어...",,남호주 최상의 품종을 선별한 우수한 와인 바로사는 강렬함과 풍성함을 잘 드러내는 쉬...,"name:울프 블라스 골드라벨 리저브 쉬라즈,pairing:각종 육류 요리, 치즈 ..."
1569,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,22000.0,R,https://github.com/audreyaiai/chatwine-dev/blo...,-1,-1,-1,-1,-1,,,,-1,"와인앤모어 AK광명점,와인앤모어 광교점,와인앤모어 동판교점,와인앤모어 삼성1호점,와...",,모스카토 로사 Brown Brothers Moscato Rosa MOSCATO 사랑...,"name:브라운 브라더스 모스카토 로사,pairing:부드러운 치즈 과일 디저트,"


In [ ]:
retriever.get_relevant_documents('{"wine_type":"레드", "body": "lt 3 gt 0"}')

#### Tool2: Search specific wine with url

In [ ]:
def search_with_url(query):
    return SeleniumURLLoader(urls=[query]).load()

#### Tool3: Wine database 2

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

#### Tool4: Search in Google

In [ ]:
search = SerpAPIWrapper()

In [ ]:
tools = [
    Tool(
        name="Wine database",
        func=retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- body: 1-5 rating int for the body of wine. You have to specify greater than or less than. For example, if you want to search for wines with a body rating of less than 3, enter 'body: gt 0 lt 3'
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine. 입력할 때는 '와인 이름은 "비냐 조잘" 입니다' 이런 식으로 입력해주세요.
"""
    ),
    Tool(
        name = "Search specific wine with url",
        func=search_with_url,
        description="Search specific wine with url. Query must be url"
    ),
    Tool(
        name = "Wine database 2",
        func=index.query,
        description="Database about the wines in wine store. You can use this tool if you're having trouble getting information from the wine database tool above. Query must be in String"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
]